In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE21359"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE21359"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE21359.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE21359.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE21359.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association of CXCL14 in the Human Airway Epithelium with Chronic Obstructive Lung Disease and Lung Cancer"
!Series_summary	"CXCL14, a recently described chemokine constitutively expressed in various epithelia, has multiple putative roles in inflammation and carcinogenesis. Based on the knowledge that cigarette smoking and the smoking-induced disorders, such as chronic obstructive pulmonary disease (COPD) and lung cancer, are associated with inflammation, we hypothesized that the airway epithelium, the primary site of smoking-induced pathologic changes in COPD and adenocarcinoma, responds to cigarette smoking with an altered CXCL14 gene expression as a part of disease-relevant molecular phenotype. Microarray analysis with subsequent TaqMan PCR validation revealed very low constitutive CXCL14 gene expression in the airway epithelium of healthy nonsmokers (n=53) which was strongly up-regulated in healthy smokers ( n=59; p<0.001) and further increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# This dataset appears to be using Affymetrix arrays to assess gene expression, 
# so it's likely to contain gene expression data
is_gene_available = True

# 2.1 Data Availability
# Trait (COPD) information is in row 3 under "smoking status"
trait_row = 3
# Age information is in row 0
age_row = 0
# Gender information is in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait information to binary (0: non-COPD, 1: COPD)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the person has COPD
    if 'COPD' in value:
        return 1
    elif 'non-smoker' in value or 'smoker,' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age information to continuous values"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender information to binary (0: female, 1: male)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary
    sample_char_dict = {0: ['Age: 41', 'age: 35', 'age: 61', 'age: 37', 'age: 47', 'age: 38', 'age: 49', 'age: 45', 'age: 36', 'age: 46', 'age: 48', 'age: 50', 'age: 56', 'age: 59', 'age: 34', 'age: 44', 'Age: 45', 'age: 29', 'age: 42', 'Age: 47', 'age: 55', 'age: 51', 'age: 60', 'age: 52', 'age: 40', 'age: 41', 'age: 43', 'age: 31', 'age: 53', 'age: 62'], 
                        1: ['Sex: M', 'sex: M', 'sex: F', 'Sex: F'], 
                        2: ['Ethnic group: black', 'ethnic group: black', 'ethnic group: white', 'ethnic group: hispanic', 'Ethnic group: white', 'ethnic group: black/hispanic', 'ethnic group: asian'], 
                        3: ['Smoking Status: non-smoker', 'smoking status: non-smoker', 'smoking status: smoker, 21 pack-years', 'smoking status: smoker, 23 pack-years', 'smoking status: smoker, 28 pack-years', 'smoking status: smoker, 20 pack-years', 'smoking status: smoker, 38 pack-years', 'smoking status: smoker, 80 pack-years', 'smoking status: smoker, 60 pack-years', 'Smoking status: non-smoker', 'smoking status: COPD, GOLD-I, 50 pack-years', 'Smoking status: COPD, GOLD-II, 33 pack-years', 'smoking status: COPD, GOLD-II, 35 pack-years', 'smoking status: COPD, GOLD-II, 20 pack-years', 'smoking status: COPD, GOLD-I, 48 pack-years', 'smoking status: COPD, GOLD-II, 75 pack-years', 'smoking status: COPD, GOLD-II, 27 pack-years', 'smoking status: COPD, GOLD-II, 60 pack-years', 'smoking status: COPD, GOLD-III, 110 pack-years', 'smoking status: COPD, GOLD-I, 22 pack-years', 'smoking status: COPD, GOLD-I, 23 pack-years', 'smoking status: smoker, 24 pack-years', 'smoking status: smoker, 29 pack-years', 'smoking status: smoker, 45 pack-years', 'smoking status: smoker, 32 pack-years', 'smoking status: smoker, 36 pack-years', 'smoking status: smoker, 15 pack-years', 'smoking status: smoker, 22 pack-years', 'smoking status: smoker, 33 pack-years', 'smoking status: smoker, 16 pack-years']}
    
    # Convert dict to DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    clinical_data = clinical_data.transpose()
    
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to CSV
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [nan, 41.0, nan], 1: [nan, nan, 1.0], 2: [nan, nan, nan], 3: [0.0, nan, nan]}
Clinical features saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE21359.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE21359/GSE21359_series_matrix.txt.gz


Gene data shape: (54675, 135)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers shows that they are in the format of Affymetrix probe IDs (e.g., 1007_s_at)
# rather than standard human gene symbols (like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to human gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the correct columns for mapping
prob_col = "ID"  # This is the column containing probe IDs in the gene annotation
gene_col = "Gene Symbol"  # This is the column containing gene symbols

# 2. Get the gene mapping dataframe 
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Mapping sample:")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Let's normalize the gene symbols in the index
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few genes after normalization:")
print(gene_data.index[:10])

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (45782, 2)
Mapping sample:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (21278, 135)
First few rows of gene expression data:
           GSM101095    GSM101096   GSM101097   GSM101098   GSM101100  \
Gene                                                                    
A1BG       26.664608    23.200108   23.611168   20.731066   22.466328   
A1BG-AS1   38.430410    37.644897   27.018625   25.424614   33.169040   
A1CF       32.887757    27.871269   27.148127   24.667728   23.662186   
A2M       305.641970  1104.150857  162.984248  257.923840  254.933860   
A2M-AS1    20.335550    27.066956   28.000036   32.134990   39.643590   

           GSM101101   GSM101102   GSM101103   GSM101104  GSM101105  ...  \
Gene                                                                 ...   
A1BG       25.030586   25.183960   22.589863   23.743465  27.456318  ...   
A1BG-AS1   26.234987   23.183222   24.775188   20.421490  25.106298  ...   
A1CF       27.690921   32.027879   26.455109   29.115901  29.179587  ...   
A2M       162

Gene expression data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE21359.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (19845, 135)
Gene data column names (sample IDs):
Index(['GSM101095', 'GSM101096', 'GSM101097', 'GSM101098', 'GSM101100'], dtype='object')



Raw clinical data structure:
Clinical data shape: (4, 136)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM101095', 'GSM101096', 'GSM101097',
       'GSM101098'],
      dtype='object')

Sample characteristics dictionary:
{0: ['Age: 41', 'age: 35', 'age: 61', 'age: 37', 'age: 47', 'age: 38', 'age: 49', 'age: 45', 'age: 36', 'age: 46', 'age: 48', 'age: 50', 'age: 56', 'age: 59', 'age: 34', 'age: 44', 'Age: 45', 'age: 29', 'age: 42', 'Age: 47', 'age: 55', 'age: 51', 'age: 60', 'age: 52', 'age: 40', 'age: 41', 'age: 43', 'age: 31', 'age: 53', 'age: 62'], 1: ['Sex: M', 'sex: M', 'sex: F', 'Sex: F'], 2: ['Ethnic group: black', 'ethnic group: black', 'ethnic group: white', 'ethnic group: hispanic', 'Ethnic group: white', 'ethnic group: black/hispanic', 'ethnic group: asian'], 3: ['Smoking Status: non-smoker', 'smoking status: non-smoker', 'smoking status: smoker, 21 pack-years', 'smoking status: smoker, 23 pack-years', 'smoking status: smoker, 28 pack-years', 'smoking status: smok

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (135, 19846)
For the feature 'Chronic_obstructive_pulmonary_disease_(COPD)', the least common label is '1' with 23 occurrences. This represents 17.04% of the dataset.
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE21359.csv
